In [ ]:
%matplotlib inline

from js import fetch

async def get_csv(url):
    res = await fetch(url)
    text = await res.text()
    filename = 'data.csv'
    with open(filename, 'w') as f:
        f.write(text)

# Group 2 Research Project

## Research Question

What genetic variants are associated with schizophrenia?

## Hypothesis

Are there multiple genes associated with schizophrenia? 

## Comment

Your hypothesis is not very informative beyond your research question. 
However, this is a hypothesis-generating study (describe that in your paper), so hypotheses tend to be more generic/non-existent for these types of studies. 

### Suggestions

- Fully define the phenotype your are interested (i.e. diagnosis of schizophrenia, severity, or something else?). 
- Give an overview of existing literature, what do we already know about this?

## Dataset Provided

For your research question, I have provided a subset of data from the Pan-UK Biobank database. 
I selected the "schizophrenia" phenotype and downloaded the full set of GWAS output. 
Since this dataset is quite large (several GB with millions of rows of data), I subsetted for variants with meta analysis (i.e. all populations) p value of <= 0.005 and only columns associated with the meta analysis. 

This contains data for 680 cases and 426,487 controls. 

The dataset contains the following columns: 

chr: Chromosome
pos: Position 
ref: Reference Allele
alt: Alt Allele
af_cases_meta: Minor Allele Frequency in Cases
af_controls_meta: Minor Allele Frequency in Controls
beta_meta: Regression coefficient for variant
se_meta: Standard error for beta estimate
pval_meta: p value for variant **Note that this is log transformed**
    - Get the p value by taking 10^p_value

### Statistician Consult

The GWAS has been completed - you need to look for the variants that are most strongly associated. 
Most likely, you'll want to filter the table for variants with p values that are below the threshold for genome wide significance (5*10^-8). 

For visualization, I've made a function for you in the next cell that will make a Manhattan plot. 
However, you'll need to add a column called "minuslog10pvalue" before you use this function. 
Remember that the p values in the dataset are already log10 transformed...

Your analysis should focus on potential biological mechanisms relating to the most strongly associated variants in the dataset. 

Your dataset can be accessed with the following url: 

https://raw.githubusercontent.com/sadams-teaching/PGPM-503-ENV/main/data/projects/group2_data.csv

In [23]:
# Function to make a manhattan plot
## Note: Your df needs a column called "minuslog10pvalue" before this function will work

def make_manhattan_plot(df):
    import matplotlib.pyplot as plt
    df["ind"] = range(len(df))
    df["chr"] = df.chr.astype('category')
    df = df.sort_values('chr')
    df_grouped = df.groupby(('chr'))

    fig = plt.figure()
    ax = fig.add_subplot(111)
    colors = ['red','green','blue', 'yellow']

    x_labels = []
    x_labels_pos = []
    for num, (name, group) in enumerate(df_grouped):
        group.plot(kind='scatter', x='ind', y='minuslog10pvalue',color=colors[num % len(colors)], ax=ax)
        x_labels.append(name)
        x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2))
    ax.set_xticks(x_labels_pos)
    ax.set_xticklabels(x_labels)
    ax.set_xlim([0, len(df)])
    ax.set_ylim([0, 20])
    ax.set_xlabel('Chromosome')
